In [1]:
!pip install wget
import os
import torch
import pandas as pd
import numpy as np
import torch
from torch import nn


ssh_config = """
Host github.com
  IdentityFile ~/.ssh/github.pem
  User davipeag
  StrictHostKeyChecking no
"""

if os.name == 'nt':
  base_path = ""
  REPO_DIR = "."
  STORE_DIR ="." 
  print("Windows")
else:
  print("Unix-like")
  REPO_DIR = "/tmp/HeartRateRegression"
  from google.colab import drive
  drive.mount('/content/drive')
  GIT_PATH = "/content/drive/My\ Drive/deeplearning_project/github.pem"
  DATA_DIR = os.path.join(REPO_DIR, "repo")
  STORE_DIR ="/content/drive/My Drive/deeplearning_project/" 
  !mkdir ~/.ssh
  !cp -u {GIT_PATH} ~/.ssh/
  !chmod u=rw,g=,o= ~/.ssh/github.pem
  !echo "{ssh_config}" > ~/.ssh/config
  !chmod u=rw,g=,o= ~/.ssh/config
  ! (cd /tmp && git clone git@github.com:davipeag/HeartRateRegression.git)
  ! (cd {REPO_DIR} && git pull )
  import sys
  sys.path.append(REPO_DIR)

def git_pull():
  ! (cd {REPO_DIR} && git pull )

git_pull()

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=30b500cf1f36bae3c0c3f73bb044b0d81d56f7ba3b67f7a17c34ea9cf09b67bf
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Unix-like
Mounted at /content/drive
Cloning into 'HeartRateRegression'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 1030 (delta 3), reused 8 (delta 3), pack-reused 1020
Receiving objects: 100% (1030/1030), 88.31 MiB | 36.79 MiB/s, done.
Resolving deltas: 100% (671/671), done.
Already up to date.
Already up to date.


In [2]:
import random
import torch
from torch import nn
import numpy as np
args = {
    'epoch_num': 250,     # Number of epochs.
    'lr': 1.0e-3,           # Learning rate.
    'weight_decay': 10e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 0,     # Number of workers on data loader.
    'batch_size': 128,     # Mini-batch size. 128
    'batch_test': 248,     # size of test batch
    'window': 15,
    'initial_window':5,
    'clip_norm': 6.0,     # Upper limit on gradient L2 norm ###
}
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

SEED = 1234
def reset_seeds():
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

reset_seeds()

cuda


In [3]:
!pip install heartpy

     |████████████████████████████████| 1.0MB 20.6MB/s 


In [5]:
from data_utils import (FormatPPGDalia, WesadExtractor, PpgDaliaExtractor)


SUBJECTS_WESAD = [2,3,4,5,6,7,8,9,10,11,13,14,15,16,17]
SUBJECTS_DALIA = [1,2,3,4,5,6,7, 9, 10, 11, 12, 13, 14, 15]

extractor = WesadExtractor(DATA_DIR)
formatter = FormatPPGDalia(from_ecg = True)
dfs_wesad = [formatter.transform(extractor.extract_subject(i)) for i in SUBJECTS_WESAD]

dalia_extractor = PpgDaliaExtractor(DATA_DIR)
dalia_formater =  FormatPPGDalia()
dfs_dalia = [dalia_formater.transform(dalia_extractor.extract_subject(i)) for i in SUBJECTS_DALIA]

dfs_train = dfs_wesad + dfs_dalia

KeyboardInterrupt: ignored

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [12]:
git_pull()

import importlib

import PPG

from PPG import FullTrainer

importlib.reload(PPG.AttentionDefaults)
importlib.reload(PPG)
importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(PPG.Models)
importlib.reload(PPG.NoHrPceLstmModel)
importlib.reload(PPG.TrainerXY)
importlib.reload(PPG.TrainerIS)
importlib.reload(PPG.FullTrainer)


Already up to date.


<module 'PPG.FullTrainer' from '/tmp/HeartRateRegression/PPG/FullTrainer.py'>

In [ ]:
import matplotlib.pyplot as plt

import pickle

  # ensembler = SimpleEnsemble()

def compute_ensemble(results):
  ps = [v["predictions"][1].reshape(-1).numpy() for v in results]
  ys = [v["predictions"][0].reshape(-1).numpy() for v in results]

  for i in range(1, len(ys)-1):
    assert np.all(ys[i] == ys[i-1])

  s = ps[0]
  for p in ps[1:]:
    s = s + p

  a = s/len(ps)
  y = ys[0]

  plt.plot(a)
  plt.plot(y)

  return np.mean(np.abs(a - y))


fchoice = {'val_sub': 4,
  'ts_sub': 0,
  'batch_size': 64,
  'weight_decay': 0,
  'lr': 0.0001,
  'lin_dropout': 0,
  'lin_size': 16,
  'nlin_layers': 2,
  'feedforward_expansion': 1,
  'nhead': 4,
  'ndec_layers': 2,
  'nenc_layers': 2,
  'conv_dropout': 0,
  'nconv_layers': 2,
  'conv_filters': 128,
  'nfeatures': 4
}


from PPG import UtilitiesDataXY
from collections import defaultdict

aresults = defaultdict(list)

for i in range(7):
  for ts_sub in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]:
    filename = f"wesad_attention_output_ts_{ts_sub}_{i}_from_ecg_joint_single_shot.pkl"
    save_path = os.path.join(STORE_DIR, filename)
    try:
      with open(save_path , "rb") as f:
        out = pickle.load(f)
    except FileNotFoundError:
      full_trainer = FullTrainer.JointValAttentionFullTrainer(dfs_train, args["device"], nepoch=40)
    else:
      aresults[ts_sub].append(out)
      print(out["args"], out["metric"])
      continue

    try:
      fchoice["ts_sub"] = ts_sub
      out = full_trainer.train(**fchoice)
      # full_trainer.dfs = dfs_wesad
      # out = full_trainer.train(**fchoice)
      print(out["args"], out["metric"])
      aresults[ts_sub].append(out)
      
      
      with open(save_path, "wb") as f:
        pickle.dump(out, f)

    except RuntimeError as e:
      if isinstance(e, KeyboardInterrupt):
        raise e
      else:
        print("####")
        print(f"Failed: {fchoice}")
        print("###")
  print(f"TS:{compute_ensemble(dresults)}")
  aresults[ts_sub].append(dresults)

